<a href="https://colab.research.google.com/github/davrodrod/algoritmosIA/blob/master/NaiveBayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Clasificador Bayes

Fuente: https://scikit-learn.org/stable/modules/naive_bayes.html

Existen varios tipos, el primero es el Gausiano que asume que las variables X siguen distribución gausiana (su media y su desviación típica se calcula automáticamente.

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)
gnb = GaussianNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print("Number of mislabeled points out of a total %d points : %d" % (X_test.shape[0], (y_test != y_pred).sum()))

Number of mislabeled points out of a total 75 points : 4


Otras variantes:

- Multinomial Naive Bayes (MultinomialNB). Modela cada entrada como un polinomio en el que las vbles son las features y = theta1*feature1 + theta2*feature2 + .. + thetan*featuren

- Complement Naive Bayes (ComplementNB). Adaptación el MNB

- Bernoulli Naive Bayes (BernoulliNB ). Los datos tienen distribución multivariable Bernuilli

- CategoricalNB Categorical distribution

Si existen demasiados datos tiene un partial_fit para entrenar poco a poco.
